In [9]:
import pandas as pd

res1 = pd.read_csv("submission.csv") #9785   drop out 
res2 = pd.read_csv("nbsvm_10foldave.csv")  #9774
res3 = pd.read_csv("rf_char.csv") # 
res4 = pd.read_csv("xgb_tfidf_10foldave.csv")  # 9799 
res5 = pd.read_csv("bigru_conv1d_bestmodel_10foldsave.csv") #9853   res5 is highly correlated with res11, so should be dropout for now
#res6 = pd.read_csv("bigru_preprocesing_addhf.csv")   #9829   drop out
res7 = pd.read_csv("tuned_lr_10foldave.csv") # 9802  
res8 = pd.read_csv("textcnn_bestmodel.csv") #9832 10foldave 
res9 = pd.read_csv("weighted_ensemble_315.csv") #9864
#res10 = pd.read_csv("xgb_stacking_319_v1.csv.csv") #9871  add meta features 
res11 = pd.read_csv("2bigru_bestmodel_10foldave.csv") #9857 10fold ave  9848 single 
res12 = pd.read_csv("rank_ave_3_19.csv") # 9861
res13 = pd.read_csv("ave_ensemble_on_blends.csv") # 9867
res14 = pd.read_csv("glove_nn_bestmodel_10foldave.csv") #9494
res15 = pd.read_csv("2layer_textcnn_bestmodel_10foldave.csv") #9829
res16 = pd.read_csv("charcnn_bestmodel_10foldave.csv") #9776
res17 = pd.read_csv("bigru_attention_bestmodel_10foldave.csv") #9848

res18 = pd.read_csv("lgb_stacking_final.csv") #9870

res19 = pd.read_csv("bigru_conv_10folds_rankave.csv") #9849
res20 = pd.read_csv("2bigru_bestmodel_10folds_rankave.csv") #9847
res21 = pd.read_csv("textcnn_prepro_bestmodel_10foldave.csv") #9830
res22 = pd.read_csv("2bigru_prepro_10foldsave.csv") #9855 
res23 = pd.read_csv("2bigru_prepro_nocu_10foldsave.csv")
res24 = pd.read_csv("capsuleNet_prepro_10foldave.csv")
res25 = pd.read_csv("bigru_conv_prepro_nocu_10foldsave.csv")

res26 = pd.read_csv("xgb_stacking_final.csv")
res27 = pd.read_csv("nn_stacking_final.csv")

In [23]:
corr(res26, res11)


 Class: toxic
 Pearson's correlation score: 0.986220
 Kendall's correlation score: 0.890753
 Spearman's correlation score: 0.979298
 Kolmogorov-Smirnov test:    KS-stat = 0.632270    p-value = 0.000e+00


 Class: severe_toxic
 Pearson's correlation score: 0.965728
 Kendall's correlation score: 0.563758
 Spearman's correlation score: 0.753868
 Kolmogorov-Smirnov test:    KS-stat = 0.934939    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: 0.986046
 Kendall's correlation score: 0.617625
 Spearman's correlation score: 0.782660
 Kolmogorov-Smirnov test:    KS-stat = 0.771970    p-value = 0.000e+00


 Class: threat
 Pearson's correlation score: 0.903405
 Kendall's correlation score: 0.737230
 Spearman's correlation score: 0.900936
 Kolmogorov-Smirnov test:    KS-stat = 0.988640    p-value = 0.000e+00


 Class: insult
 Pearson's correlation score: 0.986070
 Kendall's correlation score: 0.864677
 Spearman's correlation score: 0.971748
 Kolmogorov-Smirnov test:    KS-stat

In [8]:
import pandas as pd
import sys
from scipy.stats import ks_2samp

first_file = sys.argv[1]
second_file = sys.argv[2]

def corr(first_df, second_df):
    # assuming first column is `class_name_id`
    class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

    for class_name in class_names:
        # all correlations
        print('\n Class: %s' % class_name)
        print(' Pearson\'s correlation score: %0.6f' %
              first_df[class_name].corr(
                  second_df[class_name], method='pearson'))
        print(' Kendall\'s correlation score: %0.6f' %
              first_df[class_name].corr(
                  second_df[class_name], method='kendall'))
        print(' Spearman\'s correlation score: %0.6f' %
              first_df[class_name].corr(
                  second_df[class_name], method='spearman'))
        ks_stat, p_value = ks_2samp(first_df[class_name].values,
                                    second_df[class_name].values)
        print(' Kolmogorov-Smirnov test:    KS-stat = %.6f    p-value = %.3e\n'
              % (ks_stat, p_value))

methods used : weighted ensemble, ranke average, staking, blend of blends 

In [6]:
#blending of blends 

for c in res1.columns[1:]:
    res2[c] =  res18[c]*0.35 + res26[c]*0.3 + res27[c]*0.35
res2.to_csv("final_blending.csv",index=False)

In [10]:
# weighted ensemble 
res = [res2, res4, res7 ,res8, res11]
for c in res1.columns[1:]:
    res2[c] =  res2[c]*0.15 + res4[c]*0.15 + res7[c]*0.1 + res8[c]*0.2 + res11[c]*0.4
res2.to_csv("weighted_ensemble_315.csv",index=False)

In [11]:
res = [res9, res10, res12]
for c in res1.columns[1:]:
    res2[c] =  res9[c]*0.15 + res10[c]*0.4 + res12[c]*0.35
res2.to_csv("ave_ensemble_on_blends.csv", index=False)

**rank average**

In [10]:
import numpy as np
from scipy.stats import rankdata

lables = res1.columns[1:]
res = [res2,res4, res5,res7, res8,res11, res15,res16,res17]
res = [r[lables].values for r in res]
pred = np.zeros_like(res[0])

for r in res:
    for i in range(6):
        pred[:,i] =  np.add(pred[:,i], rankdata(r[:,i])/float(pred.shape[0]))
pred /= float(len(res))
#pred
res2[lables] = pred
res2.to_csv("rank_ave_3_19.csv", index=False)

In [19]:
for c in res1.columns[1:]:
    res2[c] =  res7[c]*0.05 + res8[c]*0.3 + res9[c]*0.4 + res5[c]*0.2 + res2[c]* 0.05
res2.to_csv("ensemble_3_13_v2.csv",index=False)

In [15]:
for c in res1.columns[1:]:
    res2[c] =  res7[c]*0.15 + res8[c]*0.3 + res5[c]*0.4+ res2[c]*0.15
res2.to_csv("ensemble_3_13.csv",index=False)

In [12]:
#best model for now 

for c in res1.columns[1:]:
    res2[c] =  res2[c]*0.15 + res1[c]*0.15 + res3[c]*0.1+ res4[c]*0 + res5[c]*0.4 + res6[c]*0.2
res2.to_csv("ensemble_3_12_v1.csv",index=False)

In [21]:
for c in res1.columns[1:]:
    res2[c] =  res2[c]*0.15 + res7[c]*0.15 + res3[c]*0.1+ res4[c]*0 + res5[c]*0.4 + res8[c]*0.2
res2.to_csv("ensemble_3_12_v3.csv",index=False)

In [17]:
import numpy as np
res = [res9, res10,res12,res13 ]
corr = np.zeros((len(res),len(res)))
for i,r in enumerate(res):
    for j,p in enumerate(res):
        corr[i][j] = np.mean(r.corrwith(p))
corr

array([[1.        , 0.96473775, 0.53917881, 0.92790516],
       [0.96473775, 1.        , 0.542977  , 0.9135585 ],
       [0.53917881, 0.542977  , 1.        , 0.7703269 ],
       [0.92790516, 0.9135585 , 0.7703269 , 1.        ]])

In [4]:
import pandas as pd
import sys
from scipy.stats import ks_2samp

first_file = sys.argv[1]
second_file = sys.argv[2]

def corr(first_df, second_df):
    # assuming first column is `class_name_id`
    class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

    for class_name in class_names:
        # all correlations
        print('\n Class: %s' % class_name)
        print(' Pearson\'s correlation score: %0.6f' %
              first_df[class_name].corr(
                  second_df[class_name], method='pearson'))
        print(' Kendall\'s correlation score: %0.6f' %
              first_df[class_name].corr(
                  second_df[class_name], method='kendall'))
        print(' Spearman\'s correlation score: %0.6f' %
              first_df[class_name].corr(
                  second_df[class_name], method='spearman'))
        ks_stat, p_value = ks_2samp(first_df[class_name].values,
                                    second_df[class_name].values)
        print(' Kolmogorov-Smirnov test:    KS-stat = %.6f    p-value = %.3e\n'
              % (ks_stat, p_value))

corr(res27, res18)


 Class: toxic
 Pearson's correlation score: 0.968033
 Kendall's correlation score: 0.879963
 Spearman's correlation score: 0.976733
 Kolmogorov-Smirnov test:    KS-stat = 0.458123    p-value = 0.000e+00


 Class: severe_toxic
 Pearson's correlation score: 0.922573
 Kendall's correlation score: 0.773755
 Spearman's correlation score: 0.924017
 Kolmogorov-Smirnov test:    KS-stat = 0.875408    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: 0.988933
 Kendall's correlation score: 0.864823
 Spearman's correlation score: 0.970545
 Kolmogorov-Smirnov test:    KS-stat = 0.539004    p-value = 0.000e+00


 Class: threat
 Pearson's correlation score: 0.783640
 Kendall's correlation score: 0.303434
 Spearman's correlation score: 0.421552
 Kolmogorov-Smirnov test:    KS-stat = 0.705688    p-value = 0.000e+00


 Class: insult
 Pearson's correlation score: 0.977897
 Kendall's correlation score: 0.885231
 Spearman's correlation score: 0.979941
 Kolmogorov-Smirnov test:    KS-stat

In [9]:
for c in res1.columns[1:]:
    res2[c] =  res2[c]*0.15 + res1[c]*0.1 + res3[c]*0.5+ res4[c]*0.1 + res5[c]*0.2 + res6[c]*0.3
res2.to_csv("ensemble_3_6_v2.csv",index=False)

In [6]:
for c in res1.columns[1:]:
    res2[c] = res2[c]*0.1 + res1[c]*0.1 + res3[c]*0.1 + res5[c]*0.1 + res6[c]*0.3 + res7[c]*0.3
res2.to_csv("ensemble_3_6.csv",index=False)

In [12]:
for c in res1.columns[1:]:
    res2[c] = res2[c]*0.1 + res1[c]*0.1 + res3[c]*0.1 + res5[c]*0.1 + res6[c]*0.3 + res7[c]*0.3
res2.to_csv("ensemble_3_7.csv",index=False)

In [24]:
for c in res1.columns[1:]:
    res2[c] = res2[c]*0.1 + res1[c]*0.1 + res3[c]*0.05 + res4[c]*0.05 + res6[c]*0.2 + res7[c]*0.2 + res8[c]*0.3
res2.to_csv("ensemble_3_8.csv",index=False)

In [27]:
for c in res1.columns[1:]:
    res2[c] = res6[c]*0.15 + res7[c]*0.35 + res8[c]*0.5
res2.to_csv("ensemble_3_8.csv",index=False)

In [30]:
for c in res1.columns[1:]:
    res2[c] = res6[c] * 0.1 +res8[c]*0.4 + res9[c]*0.5
res2.to_csv("ensemble_3_11.csv",index=False)